# Files & imports

In [1]:
from pandas import read_csv, DataFrame
import pandas as pd

In [2]:
## https://www.kaggle.com/datasets/mkechinov/direct-messaging/data #

filename_first_purchase = "dataset/client_first_purchase_date.csv"
filename_campaigns = "dataset/campaigns.csv"

filename_msg = "dataset/messages-demo.csv"
file_tag = "ecom_msg"

data_first_purchase = pd.read_csv(filename_first_purchase)
data_campaigns = pd.read_csv(filename_campaigns)
data_msg = pd.read_csv(filename_msg)

data_first_purchase.info()

/var/folders/cb/3dkrbpbn63zcqbvg5csx6rhc0000gp/T/ipykernel_67683/1758330304.py:11: DtypeWarning: Columns (7,8,16,17,19,21,23,25,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data_msg = pd.read_csv(filename_msg)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1854736 entries, 0 to 1854735
Data columns (total 2 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   client_id            int64 
 1   first_purchase_date  object
dtypes: int64(1), object(1)
memory usage: 28.3+ MB


# Load File

### >> method 1.1 - random sample on the merge [used]

In [3]:
import pandas as pd

# Sample each dataset
#data_first_purchase_sampled = sample_data(data_first_purchase, test_data=test)  # Replace test_data=False as needed
#data_campaigns_sampled = sample_data(data_campaigns, test_data=test)
#data_msg_sampled = sample_data(data_msg, test_data=test)

test = False 

def sample_per_day(group, fraction=0.1):
    return group.sample(frac=fraction)

if test==True: 

    sample = 0.01

    #data_first_purchase['first_purchase_date'] = pd.to_datetime(data_first_purchase['first_purchase_date']) 
    # Apply the sampling to each group (grouped by event_date) 1%
    #data_first_purchase = data_first_purchase.groupby('first_purchase_date').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True)

    #data_campaigns['started_at'] = pd.to_datetime(data_campaigns['started_at']) 
    # Apply the sampling to each group (grouped by event_date) 1%
    #data_campaigns = data_campaigns.groupby('started_at').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True)

    data_msg['sent_at'] = pd.to_datetime(data_msg['sent_at']) 
    # Apply the sampling to each group (grouped by event_date) 1%
    data_msg = data_msg.groupby('sent_at').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True) 
   

else: 
    sample = 0.5

    #data_first_purchase['first_purchase_date'] = pd.to_datetime(data_first_purchase['first_purchase_date']) 
    # Apply the sampling to each group (grouped by event_date) 10%
    #data_first_purchase = data_first_purchase.groupby('first_purchase_date').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True)
    
    #data_campaigns['started_at'] = pd.to_datetime(data_campaigns['started_at']) 
    # Apply the sampling to each group (grouped by event_date) 1%
    #data_campaigns = data_campaigns.groupby('started_at').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True)

    data_msg['sent_at'] = pd.to_datetime(data_msg['sent_at']) 
    # Apply the sampling to each group (grouped by event_date) 1%
    data_msg = data_msg.groupby('sent_at').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True) 

# Display some rows from the final dataset
data_first_purchase.info()
data_campaigns.info()
data_msg.info()


/var/folders/cb/3dkrbpbn63zcqbvg5csx6rhc0000gp/T/ipykernel_67683/170292560.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_msg = data_msg.groupby('sent_at').apply(lambda x: sample_per_day(x, sample)).reset_index(drop=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1854736 entries, 0 to 1854735
Data columns (total 2 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   client_id            int64 
 1   first_purchase_date  object
dtypes: int64(1), object(1)
memory usage: 28.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1907 entries, 0 to 1906
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            1907 non-null   int64  
 1   campaign_type                 1907 non-null   object 
 2   channel                       1907 non-null   object 
 3   topic                         1877 non-null   object 
 4   started_at                    1824 non-null   object 
 5   finished_at                   1808 non-null   object 
 6   total_count                   1824 non-null   float64
 7   ab_test                       12 non-null     object 
 8   war

In [4]:
# Filter the relevant columns
first_purchase_filtered = data_first_purchase[['client_id', 'first_purchase_date']]
campaign_filtered = data_campaigns[['id', 'campaign_type', 'channel', 'topic', 'started_at',
                                            'finished_at', 'total_count', 'subject_length',
                                            'subject_with_personalization', 'subject_with_deadline',
                                            'subject_with_emoji', 'subject_with_bonuses',
                                            'subject_with_discount', 'subject_with_saleout']]

# Merge datasets using left join (or inner join if desired)
data_1 = pd.merge(data_msg, first_purchase_filtered, on='client_id', how='left')

# Create composite keys
data_1['composite_key'] = data_1['campaign_id'].astype(str) + '_' + data_1['message_type'].astype(str)
campaign_filtered['composite_key'] = campaign_filtered['id'].astype(str) + '_' + campaign_filtered['campaign_type'].astype(str)

# Perform the left join using the composite key
data_final = pd.merge(data_1, campaign_filtered, on='composite_key', how='left', suffixes=('_msg', '_cmp'))


# Ensure the columns have the same data type
#data_1['campaign_id'] = data_1['campaign_id'].astype(str)
#data_1['message_type'] = data_1['message_type'].astype(str)
#
#campaign_filtered['id'] = campaign_filtered['id'].astype(str)
#campaign_filtered['campaign_type'] = campaign_filtered['campaign_type'].astype(str)

#data_final = pd.merge(data_1, campaign_filtered, left_on=['campaign_id', "message_type"], right_on=['id', "campaign_type"], how='left', suffixes=('_msg', '_cmp'))

data_final.info()

/var/folders/cb/3dkrbpbn63zcqbvg5csx6rhc0000gp/T/ipykernel_67683/316927410.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_filtered['composite_key'] = campaign_filtered['id'].astype(str) + '_' + campaign_filtered['campaign_type'].astype(str)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4831549 entries, 0 to 4831548
Data columns (total 48 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   id_msg                        int64         
 1   message_id                    object        
 2   campaign_id                   int64         
 3   message_type                  object        
 4   client_id                     int64         
 5   channel_msg                   object        
 6   category                      float64       
 7   platform                      object        
 8   email_provider                object        
 9   stream                        object        
 10  date                          object        
 11  sent_at                       datetime64[ns]
 12  is_opened                     object        
 13  opened_first_time_at          object        
 14  opened_last_time_at           object        
 15  is_clicked                    ob

In [5]:
%run "scripts/dslabs_functions.py"

python(70710) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


dslabs lodaded


# Transformations

## Remove columns

### Remove IDs

In [6]:
#drop id columns - are not relevant for the prediction of the class

data_final = data_final.drop(columns=['message_id', 'id_msg', "id_cmp", 'campaign_id','client_id'])

# Drop the composite key if it's no longer needed
data_final.drop(columns=['composite_key'], inplace=True)

### Irrelevant for business

In [7]:
#drop unnecessary values to the business/problem:

data_final = data_final.drop(columns=["created_at", "updated_at", "date", "opened_last_time_at", "clicked_last_time_at",
                               "hard_bounced_at", "soft_bounced_at", "created_at", "updated_at", "finished_at", "channel_msg"
                          ])

#created_at and updated_at, not necessary

### High MV (80%+)

In [8]:
#drop MV columns - where MV values are around 90% or above. 

data_final = data_final.drop(columns=["category", "stream", "blocked_at", "is_blocked", "message_type"
                          ])

#stream tem apenas 1 valor e nao ajuda com nada ao modelo
#category, blocked_at tem 100% MV
#is_blocked: 1 distinct values
#message_type dropped, same as campaign_type


# Split 'date' column

In [9]:
import pandas as pd

# Split the 'date'' column to have more information and detail on the:
# - day_of_month
# - day_ok_week 
# - is_weekend
# - moment_of_day
# - hour
# - min
#As the dataset only has data from sepecific month of 2021, we don't need to get the year, quarter.

# Convert the 'date' column to datetime
data_final['sent_at'] = pd.to_datetime(data_final['sent_at'], utc=True)

# Create new columns based on the date column 'event_time'
data_final['day_of_year'] = data_final['sent_at'].dt.dayofyear  # Day of the year
data_final['week_of_month'] = data_final['sent_at'].apply(lambda x: (x.day - 1) // 7 + 1)
data_final['is_weekend'] = data_final['sent_at'].dt.weekday.apply(lambda x: True if x >= 5 else False)
data_final['day_of_week_nr'] = data_final['sent_at'].dt.weekday  # Monday=0, Sunday=6
data_final['day_of_month'] = data_final['sent_at'].dt.day
data_final['month'] = data_final['sent_at'].dt.month_name()
data_final['hour'] = data_final['sent_at'].dt.hour
data_final['min'] = data_final['sent_at'].dt.minute

# Create function to get time of the day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

data_final['time_of_day'] = data_final['hour'].apply(categorize_time_of_day)

# Display the updated DataFrame
data_final.head()


,platform,email_provider,sent_at,is_opened,opened_first_time_at,is_clicked,clicked_first_time_at,is_unsubscribed,unsubscribed_at,is_hard_bounced,...,subject_with_saleout,day_of_year,week_of_month,is_weekend,day_of_week_nr,day_of_month,month,hour,min,time_of_day
0,NaN,yandex.ru,2021-04-30 00:24:18+00:00,t,2021-05-17 13:11:58,f,NaN,f,NaN,f,...,False,120,5,False,4,30,April,0,24,Night
1,NaN,mail.ru,2021-04-30 00:30:28+00:00,f,NaN,f,NaN,f,NaN,f,...,False,120,5,False,4,30,April,0,30,Night
2,NaN,mail.ru,2021-04-30 01:11:21+00:00,t,2021-04-30 03:29:57,f,NaN,f,NaN,f,...,False,120,5,False,4,30,April,1,11,Night
3,NaN,mail.ru,2021-04-30 01:11:21+00:00,f,NaN,f,NaN,f,NaN,f,...,False,120,5,False,4,30,April,1,11,Night
4,NaN,gmail.com,2021-04-30 01:17:37+00:00,f,NaN,f,NaN,f,NaN,f,...,False,120,5,False,4,30,April,1,17,Night


# Can "is_clicked" happen at the same time as other events?
['is_opened', 'is_unsubscribed', 'is_complained', 'is_purchased']
- Yes it can, however, whenever those variables are T, my target will always be T.
    - This means i have to remove them from the dataset.

(opened_first_time_at, clicked_first_time_at, unsubscribed_at, complained_at, purchased_at, first_purchase_date, started_at)
- same thing as above
    - This means i have to remove them from the dataset.

In [10]:
import pandas as pd

# Assuming your DataFrame is named 'data_final'
# Create a dictionary to store the counts of simultaneous occurrences
results = {}

# List of binary variables to compare with 'is_clicked'
variables_to_compare = ['is_opened', 'is_unsubscribed', 'is_complained', 'is_purchased', "is_clicked"]

# Loop through each variable and compare with 'is_clicked'
for variable in variables_to_compare:
    count_simultaneous = len(data_final[(data_final['is_clicked'] == "t") & (data_final[variable] == "t")])
    results[variable] = count_simultaneous

# Display the results
print(results)


{'is_opened': 105487, 'is_unsubscribed': 599, 'is_complained': 112, 'is_purchased': 5004, 'is_clicked': 105488}


In [11]:
import pandas as pd

# Assuming your DataFrame is named 'data_final'
# Create a dictionary to store the counts of simultaneous occurrences where date fields are not null
results = {}

# List of date variables to check against 'is_clicked'
date_variables = ['opened_first_time_at', 'clicked_first_time_at', 'unsubscribed_at', 
                  'complained_at', 'purchased_at', 'first_purchase_date', 'started_at']

# Loop through each date variable and check if 'is_clicked' is True and the date variable is not null
for date_var in date_variables:
    count_simultaneous = len(data_final[(data_final['is_clicked'] == "t") & (data_final[date_var].notna())])
    results[date_var] = count_simultaneous

# Display the results
print(results)


{'opened_first_time_at': 105487, 'clicked_first_time_at': 105488, 'unsubscribed_at': 599, 'complained_at': 112, 'purchased_at': 5004, 'first_purchase_date': 41225, 'started_at': 43218}


In [12]:
#remove other date columns:

data_final = data_final.drop(columns=['is_opened', 'is_unsubscribed', 'is_complained', 'is_purchased',
                                       'opened_first_time_at', 'clicked_first_time_at', 'unsubscribed_at', 
                  'complained_at', 'purchased_at', 'first_purchase_date', 'started_at', "is_hard_bounced", "is_soft_bounced"])

In [13]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4831549 entries, 0 to 4831548
Data columns (total 24 columns):
 #   Column                        Dtype              
---  ------                        -----              
 0   platform                      object             
 1   email_provider                object             
 2   sent_at                       datetime64[ns, UTC]
 3   is_clicked                    object             
 4   campaign_type                 object             
 5   channel_cmp                   object             
 6   topic                         object             
 7   total_count                   float64            
 8   subject_length                float64            
 9   subject_with_personalization  object             
 10  subject_with_deadline         object             
 11  subject_with_emoji            object             
 12  subject_with_bonuses          object             
 13  subject_with_discount         object             
 14  su

## Remove date columns
Date Variables:
-  🚫date: 20 distinct values
-  ✅sent_at: 35261 distinct values
-  ✅opened_first_time_at: 8720 distinct values
-  🚫opened_last_time_at: 8737 distinct values
-  ✅clicked_first_time_at: 892 distinct values
-  🚫clicked_last_time_at: 893 distinct values
-  ✅unsubscribed_at: 1923 distinct values
-  🚫hard_bounced_at: 4 distinct values
-  🚫soft_bounced_at: 12 distinct values
-  ✅complained_at: 19 distinct values
-  ✅purchased_at: 21 distinct values
-  🚫created_at: 71568 distinct values
-  🚫updated_at: 71568 distinct values
-  ✅first_purchase_date: 730 distinct values
-  ✅started_at: 35 distinct values - Bulk campaign start datetime
-  🚫finished_at: 35 distinct values - Bulk campaign finish datetime

Creating more variables considering "sent_at"

In [14]:
## Convert all date columns to datetime format
#date_columns = ['opened_first_time_at', 'clicked_first_time_at', 'unsubscribed_at', 
#                'complained_at', 'purchased_at', 'first_purchase_date', 'started_at']
#
#data_final[['sent_at'] + date_columns] = data_final[['sent_at'] + date_columns].apply(pd.to_datetime)
#
## Remove timezone information if any column has it, making all columns tz-naive
#for col in ['sent_at'] + date_columns:
#    data_final[col] = data_final[col].dt.tz_localize(None)
#
## Calculate the difference in days between 'sent_at' and each of the specified date columns
#for col in date_columns:
#    data_final[f'days_diff_{col}_sent_at'] = (data_final[col] - data_final['sent_at']).dt.days
#
## Display the updated DataFrame
#data_final.head()

In [15]:
#remove other date columns:

#data_final = data_final.drop(columns=['opened_first_time_at', 'clicked_first_time_at', 'unsubscribed_at', 
#                'complained_at', 'purchased_at', 'first_purchase_date', 'started_at'])


# Save df from preprocessing into a csv file

In [16]:
# Definir o caminho para salvar o arquivo CSV
filepath = r'dataset/df_msg_pre_proc.csv'

# Salvar o DataFrame em CSV
data_final.to_csv(filepath, index=False)

data_final.head()


,platform,email_provider,sent_at,is_clicked,campaign_type,channel_cmp,topic,total_count,subject_length,subject_with_personalization,...,subject_with_saleout,day_of_year,week_of_month,is_weekend,day_of_week_nr,day_of_month,month,hour,min,time_of_day
0,NaN,yandex.ru,2021-04-30 00:24:18+00:00,f,transactional,email,order created,NaN,74.0,False,...,False,120,5,False,4,30,April,0,24,Night
1,NaN,mail.ru,2021-04-30 00:30:28+00:00,f,transactional,email,order created,NaN,74.0,False,...,False,120,5,False,4,30,April,0,30,Night
2,NaN,mail.ru,2021-04-30 01:11:21+00:00,f,transactional,email,order created,NaN,74.0,False,...,False,120,5,False,4,30,April,1,11,Night
3,NaN,mail.ru,2021-04-30 01:11:21+00:00,f,transactional,email,order created,NaN,74.0,False,...,False,120,5,False,4,30,April,1,11,Night
4,NaN,gmail.com,2021-04-30 01:17:37+00:00,f,transactional,email,order ready for pickup,NaN,88.0,False,...,False,120,5,False,4,30,April,1,17,Night


In [17]:
data_final.shape[0]

4831549